# Financial Model Prediction

In [1]:
import pandas as pd
import numpy as np
import seaborn as sbn
import re

import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import classification_report
from sklearn.decomposition import PCA

import lightgbm as lgb

%matplotlib inline

C:\Users\esteb\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
def add_shifts(df_, col_to_shift, new_col, shift):
    """
    This function add shifted columns to data by ticker.
    
    :param pd.DataFrame df_: Dataframe with financial data.
    :param str col_to_shift: Column over to create the shift.
    :param str new_col: Name of the shifted column.
    :param int shift: Days to use as shift.
    :return pd.DataFrame: Dataframe with the shift added.
    """
    
    for id_ in df_['ticker'].unique():
        df_by_id = df_[df_['ticker'] == id_]
        df_.loc[df_['ticker'] == id_, new_col] = df_by_id[col_to_shift] - df_by_id[col_to_shift].shift(shift)
        
    return df_

In [3]:
def get_non_n_cols(df_, n):
    """
    Get the columns that his window is not n days
    
    :param pd.DataFrame df_: Dataframe with financial data.
    :param int n: n days to get columns with.
    :return list: List with the name of the columns.
    """
    
    return [elem for elem in df_.columns if (re.search(r'\d+$', elem) is not None) and (int(elem[-2:].strip()) != n)]

In [4]:
def interpolate_nan_values(df_, to_interpolate):
    """
    Interpolate and extrapolate nan values for numerical columns.
    
    :param pd.DataFrame df_: Dataframe with financial data with NaN values.
    :param list to_interpolate: List with columns to interpolate.
    :return pd.DataFrame: Dataframe with financial data without NaN values.
    """
    
    list_df = []
    for tick in df_['ticker'].unique():
        df_by_ticker = df_[df_['ticker'] == tick]
        for col in to_interpolate:
            df_by_ticker[col] = df_by_ticker[col].interpolate(method='linear', limit_direction='both')
        list_df.append(df_by_ticker)
    return pd.concat(list_df)

In [5]:
df_categorical = pd.read_csv('../data/db_bsm_categorical.csv')
df_financial = pd.read_csv('../data/db_bsm_financial.csv')

In [18]:
df_financial.replace(0, np.NaN, inplace=True)
df_financial.isnull().sum().to_frame('Null Values').loc[['close', 'volume']]

,Null Values
close,1081
volume,1996


In [16]:
df_financial_not_nan = interpolate_nan_values(df_financial, ['close', 'volume'])

In [19]:
df_financial_not_nan.isnull().sum().to_frame('Null Values').loc[['close', 'volume']]

,Null Values
close,0
volume,0


In [21]:
num_list = [3, 5, 7, 14, 21]

In [22]:
for num_ in num_list:
    df_fin = add_shifts(df_financial_not_nan, 'close', 'close_shifted_%i' % num_, num_)

In [32]:
df_fin.dropna(subset=['close_shifted_21'], inplace=True)

In [47]:
df_fin_not_nan = interpolate_nan_values(df_fin, list(df_fin.select_dtypes(float)))

In [48]:
df_fin_not_nan.isnull().sum().to_frame('Null Values').sort_values(by='Null Values', ascending=False).head(3)

,Null Values
ADX 14,0
RSI 21,0
ROCR 14,0


In [55]:
df_fin_not_nan['date'] = pd.to_datetime(df_fin_not_nan['date'])

In [73]:
df_fin_not_nan['year'], df_fin_not_nan['month'] = df_fin_not_nan['date'].dt.year, df_fin_not_nan['date'].dt.month
df_fin_not_nan.loc[df_fin_not_nan['close_shifted_3'] >= 0, 'sign'] = '+'
df_fin_not_nan.loc[df_fin_not_nan['close_shifted_3'] < 0, 'sign'] = '-'

In [74]:
df_fin_not_nan.groupby(['ticker', 'year', 'month', 'sign'])[['close_shifted_3']].describe()

close_shifted_3                                     \
                                 count      mean       std    min     25%   
ticker year month sign                                                      
AAL    2006 2     -                2.0 -0.925000  0.898026  -1.56 -1.2425   
            3     +               13.0  1.859231  0.829623   0.85  1.3300   
                  -               10.0 -0.893000  0.471641  -1.68 -1.1650   
            4     +               12.0  1.955833  1.710654   0.17  0.6475   
                  -                7.0 -1.160000  0.959896  -2.65 -1.8550   
            5     +               12.0  2.161667  1.473998   0.37  1.0225   
                  -               10.0 -1.881000  1.301430  -3.61 -3.0375   
            6     +               14.0  2.178571  1.248765   0.03  1.2325   
                  -                8.0 -2.778750  1.917233  -5.63 -3.8600   
            7     +               10.0  3.041000  1.764051   0.28  2.3525   
                  -               10.0 -3.447000  2.389989  -7.33 -5.3250   
            8     +               10.0  2.173000  1.951865   0.08  0.8825   
                  -               13.0 -2.740000  1.614027  -4.76 -4.3300   
            9     +               10.0  1.965000  1.961463   0.02  0.5225   
                  -               10.0 -1.410000  1.243356  -3.67 -1.6775   
            10    +               14.0  2.032143  1.570395   0.09  0.4850   
                  -                8.0 -1.437500  0.963265  -2.75 -2.3800   
            11    +               12.0  3.819167  3.447575   0.28  1.8900   
                  -                9.0 -2.736667  1.818103  -6.19 -3.7800   
            12    +               12.0  1.065000  1.123092   0.00  0.4200   
                  -                8.0 -2.228750  1.430339  -4.48 -3.0825   
       2007 1     +                9.0  2.658889  1.273288   0.64  1.6300   
                  -               11.0 -2.460909  1.530160  -4.94 -3.4200   
            2     +                7.0  1.855714  1.818863   0.10  0.5600   
                  -               12.0 -1.590000  0.920869  -2.83 -2.1950   
            3     +                5.0  1.446000  1.319462   0.04  0.1000   
                  -               17.0 -1.601765  0.723042  -3.15 -1.8700   
            4     +                7.0  1.334286  0.871796   0.08  0.6700   
                  -               13.0 -2.562308  1.886404  -5.71 -3.7700   
            5     +                8.0  1.091250  0.665892   0.03  0.6200   
...                                ...       ...       ...    ...     ...   
XOM    2018 1     +               14.0  1.097143  0.904335   0.12  0.5050   
                  -                7.0 -0.690000  0.690338  -1.70 -1.1300   
            2     +                7.0  1.368571  1.293966   0.23  0.4600   
                  -               12.0 -3.299167  3.445502 -10.72 -4.6325   
            3     +                8.0  0.606250  0.265165   0.20  0.5000   
                  -               13.0 -1.216154  0.967880  -3.64 -1.6200   
            4     +               16.0  1.205625  0.853823   0.00  0.6275   
                  -                5.0 -0.802000  0.624075  -1.75 -1.0700   
            5     +               13.0  1.563077  1.277715   0.05  0.7400   
                  -                9.0 -1.728889  1.486140  -3.91 -2.9300   
            6     +               10.0  1.579000  1.096433   0.23  0.7475   
                  -               11.0 -0.958182  0.579134  -2.09 -1.2000   
            7     +                9.0  1.021111  0.948926   0.05  0.2700   
                  -               12.0 -0.869167  0.763633  -2.73 -1.0900   
            8     +               12.0  0.873333  0.599520   0.11  0.3975   
                  -               11.0 -1.282727  0.894473  -2.48 -1.9300   
            9     +               16.0  1.119375  0.694363   0.06  0.5450   
                  -                3.0 -0.960000  0.468722  -1.48 -1.1550   
            10    +